In [1]:
import tensorflow as tf


# Check if GPU is available
gpu_available = 1 if tf.config.list_physical_devices('GPU') else 0
print(gpu_available)


2024-02-22 15:34:34.182437: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 15:34:34.217095: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 15:34:34.217119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 15:34:34.217927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 15:34:34.223477: I tensorflow/core/platform/cpu_feature_guar

1


2024-02-22 15:34:35.734373: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 15:34:35.764150: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 15:34:35.764312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import os
import cv2
import numpy as np
from osgeo import gdal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pennylane as qml
from qiskit import Aer, execute, QuantumCircuit
from qiskit.circuit import QuantumCircuit, Parameter
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence

In [3]:
dataset_path = "/home/bharatforge/Selvin/EuroSAT_MS"

In [4]:
subdirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Initialize lists to store images and labels
images = []
labels = []

# Initialize a label counter
label = 0

# Loop through each subdirectory
for subdir in subdirs:
    # List all image files in the subdirectory
    image_files = [os.path.join(dataset_path, subdir, f) for f in os.listdir(os.path.join(dataset_path, subdir)) if f.endswith(".tif")]
    
    # Load images from the files
    for file in image_files:
        # Open the image file using GDAL
        dataset = gdal.Open(file)

        # Read all image channels into a list
        channels = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]

        # Stack the channels to create a multi-channel image
        image = np.stack(channels, axis=-1)

        # Resize the image to a fixed size (if needed)
        image = cv2.resize(image, (64, 64))  # Adjust the dimensions as per your requirements
        
        # Append the image and label to the lists
        images.append(image)
        labels.append(label)  # Use the label counter as the label
    
    # Increment the label counter for the next subdirectory
    label += 1

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values to be between 0 and 1
images = images / 255.0

In [5]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
import tensorflow as tf
import pennylane as qml
from tensorflow.keras.layers import Layer

class Quantum(Layer):
    def __init__(self, **kwargs):
        super(Quantum, self).__init__(**kwargs)
        self.dev = qml.device("default.qubit", wires=4)

    def build(self, input_shape):
        super(Quantum, self).build(input_shape)

    def call(self, inputs):
        # Convert inputs to tensor
        inputs = tf.convert_to_tensor(inputs)
        # Compute quantum circuit results
        output = tf.vectorized_map(self.quantum_func, inputs)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 16)

    def quantum_func(self, inputs):
        @qml.qnode(self.dev, interface='tf')
        def quantum_circuit(params):
            qml.Hadamard(wires=0)
            qml.Hadamard(wires=1)
            qml.Hadamard(wires=2)
            qml.Hadamard(wires=3)

            # Use the values in the batch item for the initial rotation (theta1)
            for i in range(4):
                qml.RY(params[i], wires=i)

            return qml.probs(wires=[0, 1, 2, 3])

        # Compute quantum circuit results
        probs = quantum_circuit(inputs)
        return probs



In [7]:
# Add layers to the model
model =Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 13)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4, activation='relu'))
model.add(BatchNormalization())

# Add the Quantum layer
model.add(Quantum())

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(10, activation='softmax'))


2024-02-22 15:35:25.461121: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 15:35:25.461559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 15:35:25.461791: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

InternalError: cudaSetDevice() on GPU:0 failed. Status: out of memory

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('Quantum_best_model_no_entanglement.h5', save_best_only=True)

# Create an instance of the Adam optimizer with the specified learning rate

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, model_checkpoint])

model.save('Quantum_best_model_no_entanglement.h5')

class_names = ['SeaLake', 'Industrial', 'HerbaceousVegetation', 'PermanentCrop', 'AnnualCrop', 'Residential', 'Pasture', 'Forest', 'Highway', 'River']


Epoch 1/30


2024-01-12 19:11:10.655360: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-12 19:11:11.298004: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-12 19:11:11.507114: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-12 19:11:13.618836: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe51ac6b4d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-12 19:11:13.618872: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2024-01-12 19:11:13.625384: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705066873.739978    8059 device_compiler.h:186] 

338/338 [==============================] - 22s 31ms/step - loss: 2.0564 - accuracy: 0.2575 - val_loss: 1.9230 - val_accuracy: 0.3974
Epoch 2/30
  7/338 [..............................] - ETA: 6s - loss: 1.8118 - accuracy: 0.3817

/home/bharatforge/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


338/338 [==============================] - 8s 25ms/step - loss: 1.6453 - accuracy: 0.4419 - val_loss: 1.3708 - val_accuracy: 0.6320
Epoch 3/30
338/338 [==============================] - 8s 25ms/step - loss: 1.3707 - accuracy: 0.5955 - val_loss: 1.2087 - val_accuracy: 0.6257
Epoch 4/30
338/338 [==============================] - 8s 25ms/step - loss: 1.1733 - accuracy: 0.6562 - val_loss: 0.9698 - val_accuracy: 0.7107
Epoch 5/30
338/338 [==============================] - 8s 25ms/step - loss: 1.0171 - accuracy: 0.6966 - val_loss: 0.8400 - val_accuracy: 0.7313
Epoch 6/30
338/338 [==============================] - 8s 25ms/step - loss: 0.9030 - accuracy: 0.7205 - val_loss: 0.8613 - val_accuracy: 0.7052
Epoch 7/30
338/338 [==============================] - 8s 25ms/step - loss: 0.8117 - accuracy: 0.7457 - val_loss: 0.6435 - val_accuracy: 0.8081
Epoch 8/30
338/338 [==============================] - 8s 25ms/step - loss: 0.7263 - accuracy: 0.7820 - val_loss: 0.5599 - val_accuracy: 0.8530
Epoch 9/30

In [9]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
training_loss, training_accuracy = model.evaluate(X_train, y_train)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

print()

print("Training loss:", training_loss)
print("Training accuracy:", training_accuracy)

675/675 [==============================] - 6s 8ms/step - loss: 0.1500 - accuracy: 0.9594
Test loss: 0.31814098358154297
Test accuracy: 0.9190740585327148

Training loss: 0.15003913640975952
Training accuracy: 0.9594444632530212


In [10]:
loaded_model = load_model("Quantum_best_model_no_entanglement.h5", custom_objects={"Quantum": Quantum})

In [11]:
predictions = model.predict(X_test)

for i in range(len(predictions)):
    predicted_label = np.argmax(predictions[i])
    true_label = y_test[i]
    print(f"Predicted: {predicted_label}, True: {true_label}")

169/169 [==============================] - 2s 7ms/step
Predicted: 2, True: 2
Predicted: 6, True: 6
Predicted: 3, True: 2
Predicted: 5, True: 5
Predicted: 4, True: 4
Predicted: 1, True: 1
Predicted: 7, True: 7
Predicted: 7, True: 7
Predicted: 8, True: 8
Predicted: 9, True: 5
Predicted: 5, True: 5
Predicted: 7, True: 7
Predicted: 6, True: 6
Predicted: 1, True: 1
Predicted: 3, True: 3
Predicted: 7, True: 7
Predicted: 9, True: 9
Predicted: 9, True: 9
Predicted: 1, True: 1
Predicted: 3, True: 3
Predicted: 1, True: 1
Predicted: 5, True: 5
Predicted: 1, True: 1
Predicted: 5, True: 5
Predicted: 7, True: 7
Predicted: 8, True: 8
Predicted: 0, True: 0
Predicted: 2, True: 2
Predicted: 7, True: 7
Predicted: 6, True: 6
Predicted: 5, True: 5
Predicted: 4, True: 4
Predicted: 7, True: 7
Predicted: 8, True: 8
Predicted: 7, True: 7
Predicted: 3, True: 2
Predicted: 7, True: 7
Predicted: 1, True: 1
Predicted: 7, True: 7
Predicted: 5, True: 5
Predicted: 1, True: 1
Predicted: 8, True: 8
Predicted: 1, True: 1

In [12]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_labels))

169/169 [==============================] - 1s 7ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       375
           1       0.89      0.98      0.94       616
           2       0.96      0.85      0.90       605
           3       0.71      0.97      0.82       520
           4       0.99      0.99      0.99       611
           5       0.93      0.83      0.88       507
           6       0.97      0.96      0.97       473
           7       1.00      0.99      1.00       597
           8       0.96      0.81      0.88       591
           9       0.96      0.89      0.92       505

    accuracy                           0.92      5400
   macro avg       0.93      0.92      0.92      5400
weighted avg       0.93      0.92      0.92      5400



In [14]:
# custom_image_path = '/home/bharatforge/Selvin/EuroSAT_MS/Pasture/Pasture_15.tif'

# class_names = ['Highway', 'Pasture', 'HerbaceousVegetation', 'PermanentCrop', 'Forest', 'Residential', 'River', 'SeaLake', 'AnnualCrop', 'Industrial']


# dataset = gdal.Open(custom_image_path)

# channels = [dataset.GetRasterBand(i+1).ReadAsArray() for i in range(dataset.RasterCount)]

# image = np.stack(channels, axis=-1)

# image = cv2.resize(image, (64, 64))

# image = image/ 255.0

# image = np.expand_dims(image, axis=0)

# predictions = model.predict(image)

# predicted_label = np.argmax(predictions)

# predicted_class_name = class_names[predicted_label]

# # Print the predicted label
# print("Predicted Label:", predicted_class_name)